
# TP3 — SQLAlchemy (CRUD + Relacionamentos)

**Observação:** todos os exercícios deste TP3 devem ser feitos utilizando a biblioteca **SQLAlchemy (ORM)**.  
Opcional: use `echo=True` no `create_engine` para visualizar o SQL.


## Parte 1 — Catálogo de Filmes


### 1) Setup do banco e ORM (enunciado)

Crie o engine (ex: `sqlite:///cinema.db`), uma `declarative_base` e a classe `Filme` (tabela “filmes”) com as colunas:
- id — inteiro, chave primária, autoincremento  
- titulo — texto  
- genero — texto  
- ano — inteiro  
- preco — número decimal  
- avaliacao — número decimal (0 a 10)  

Crie as tabelas e inicialize uma seção (session).


In [3]:

# === GABARITO: Setup do banco e ORM ===
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker

# Usaremos um caminho fixo na sandbox para garantir funcionamento
engine = create_engine(f"sqlite:///cinema.db", echo=False)

Base = declarative_base()

class Filme(Base):
    __tablename__ = "filmes"
    id = Column(Integer, primary_key=True, autoincrement=True)
    titulo = Column(String, nullable=False)
    genero = Column(String, nullable=False)
    ano = Column(Integer, nullable=False)
    preco = Column(Float, nullable=False)
    avaliacao = Column(Float, nullable=False)  # 0-10

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

print("Banco criado em:", CINEMA_DB_PATH)


Banco criado em: data/cinema.db



### 2) Parsing do arquivo “filmes.txt” (enunciado)

Abra o arquivo, leia e parseie cada linha nos campos: `titulo`, `genero`, `ano`, `preco`, `avaliacao`.  
Armazene os registros em uma lista e exiba os **3 primeiros** itens.


In [5]:

# === GABARITO: Parsing do arquivo "filmes.txt" ===
from pathlib import Path

FILMES_TXT = Path("../../data/filmes.txt")

# Gerar um arquivo de exemplo caso não exista (alunos podem trocar pelo arquivo próprio)
if not FILMES_TXT.exists():
    sample = [
        "Star Voyage;Ficcao;2005;39.90;8.2",
        "Amor & Comedia;Comedia;2012;19.90;6.8",
        "Drama da Vida;Drama;2008;29.90;7.5",
        "Star Night;Ficcao;2011;49.90;8.7",
        "Ritmo de Ação;Acao;2003;59.90;7.9",
        "Pagando Promessas;Drama;1998;24.90;7.0",
        "Galaxy Star;Ficcao;2015;54.90;7.6",
        "A Comedia dos Erros;Comedia;2019;22.50;6.9",
        "Velocidade Máxima;Acao;2007;62.00;8.1",
        "Romance no Campo;Drama;2002;18.00;6.4",
    ]
    FILMES_TXT.write_text("\n".join(sample), encoding="utf-8")

# Parse simples: cada linha "titulo;genero;ano;preco;avaliacao"
registros = []
for line in FILMES_TXT.read_text(encoding="utf-8").splitlines()[1:-1]:
    titulo, genero, ano, preco, avaliacao = line.split(";")
    registros.append({
        "titulo": titulo.strip(),
        "genero": genero.strip(),
        "ano": int(ano),
        "preco": float(preco),
        "avaliacao": float(avaliacao),
    })

registros[:3]


[{'titulo': 'Interstellar',
  'genero': 'Ficção',
  'ano': 2014,
  'preco': 29.9,
  'avaliacao': 8.6},
 {'titulo': 'The Dark Knight',
  'genero': 'Ação',
  'ano': 2008,
  'preco': 24.9,
  'avaliacao': 9.0},
 {'titulo': 'La La Land',
  'genero': 'Musical',
  'ano': 2016,
  'preco': 19.9,
  'avaliacao': 8.0}]


### 3) Inserções (via ORM) (enunciado)

Insira todos os filmes do arquivo usando a tabela criada no exercício 1.  
Observação: não esqueça do commit.


In [6]:

# === GABARITO: Inserções no banco ===
# Limpar a tabela antes (reexecução idempotente do gabarito)
session.query(Filme).delete()
session.commit()

objs = [Filme(**r) for r in registros]
session.add_all(objs)
session.commit()

print("Registros inseridos:", session.query(Filme).count())


Registros inseridos: 56



### 4) Consultas: listagens, filtros e paginação (enunciado)

- Liste todos os filmes mostrando apenas título e ano, ordenados por ano crescente.  
- Liste os 5 filmes com maior avaliação, ordenados por avaliação decrescente.  
- Filtre **apenas** pelos gêneros “Drama” e “Comédia”.  
- Filtre por preço entre 20,00 e 60,00 (inclusive), ordenados por preço decrescente.  
- Pesquise por títulos que contenham “star” (case-insensitive), limite a 10 resultados, ordenados por título (A–Z).  
- **Paginação (desafio lógico):** defina uma consulta base com filmes de avaliação ≥ 7,0 e ano ≥ 2000, ordenada por avaliação de forma decrescente. Com `page_size = 3`, retorne: **Página 1**, **Página 3** e a **Última página**.


In [7]:

# === GABARITO: Consultas ===
from sqlalchemy import select, func, desc, asc, and_, or_
from sqlalchemy.orm import Session

# a) Título e ano, ordenados por ano crescente
q_a = session.query(Filme.titulo, Filme.ano).order_by(asc(Filme.ano))
a = q_a.all()

# b) Top 5 por avaliação desc
q_b = session.query(Filme.titulo, Filme.avaliacao).order_by(desc(Filme.avaliacao)).limit(5)
b = q_b.all()

# c) Apenas Drama e Comédia
q_c = session.query(Filme).filter(Filme.genero.in_(["Drama", "Comedia"]))
c = [(f.titulo, f.genero) for f in q_c.all()]

# d) Preço entre [20, 60], ordenado por preço desc
q_d = session.query(Filme.titulo, Filme.preco).filter(Filme.preco.between(20.0, 60.0)).order_by(desc(Filme.preco))
d = q_d.all()

# e) Títulos contendo "star" (case-insensitive), limit 10, ordenados por título A–Z
q_e = session.query(Filme.titulo).filter(Filme.titulo.ilike("%star%")).order_by(asc(Filme.titulo)).limit(10)
e = q_e.all()

# f) Paginação (consulta base)
base = session.query(Filme).filter(and_(Filme.avaliacao >= 7.0, Filme.ano >= 2000)).order_by(desc(Filme.avaliacao))
page_size = 3

def fetch_page(query, page, page_size):
    # page 1 -> offset 0
    return query.offset((page-1)*page_size).limit(page_size).all()

total = base.count()
last_page = (total + page_size - 1) // page_size if total else 1

page1 = fetch_page(base, 1, page_size)
page3 = fetch_page(base, 3, page_size)
lastp = fetch_page(base, last_page, page_size)

print("a) Título/Ano (ano crescente) ->", a[:10])
print("b) Top5 Avaliação ->", b)
print("c) Drama/Comédia ->", c[:10])
print("d) Preço [20,60] desc ->", d[:10])
print("e) Contém 'star' ->", e)
print("f) total, last_page ->", total, last_page)
print("   Página1 ->", [(x.titulo, x.avaliacao) for x in page1])
print("   Página3 ->", [(x.titulo, x.avaliacao) for x in page3])
print("   Última  ->", [(x.titulo, x.avaliacao) for x in lastp])


a) Título/Ano (ano crescente) -> [('Star Wars: A New Hope', 1977), ('Road House', 1989), ('Home Alone', 1990), ('The Lion King', 1994), ('Whisper of the Heart', 1995), ('Shakespeare in Love', 1998), ('Dark City', 1998), ('Fight Club', 1999), ('In the Mood for Love', 2000), ('The Lord of the Rings: The Fellowship of the Ring', 2001)]
b) Top5 Avaliação -> [('The Dark Knight', 9.0), ('The Lord of the Rings: The Return of the King', 9.0), ('The Lord of the Rings: The Two Towers', 8.8), ('The Lord of the Rings: The Fellowship of the Ring', 8.8), ('Inception', 8.8)]
c) Drama/Comédia -> [('Parasite', 'Drama'), ('Whiplash', 'Drama'), ('A Star Is Born', 'Drama'), ('Green Book', 'Drama'), ('City of God', 'Drama'), ('The King’s Speech', 'Drama'), ('Road to Perdition', 'Drama'), ('The Long Road Home', 'Drama'), ('The Imitation Game', 'Drama'), ('The Social Network', 'Drama')]
d) Preço [20,60] desc -> [('Interstellar', 29.9), ('The Lord of the Rings: The Return of the King', 28.9), ('The Lord of th


### 5) Agregações e verificação de existência (enunciado)

- Conte quantos filmes existem no total.  
- Calcule o preço médio de todos os filmes.  
- Calcule o preço médio por gênero.  
- Busque um filme pelo id.  
- Verifique se existe algum filme de ficção.


In [8]:
# # === GABARITO: Agregações e verificação ===

total_filmes = session.query(func.count(Filme.id)).scalar()
preco_medio = session.query(func.avg(Filme.preco)).scalar()
preco_medio_por_genero = session.query(Filme.genero, func.avg(Filme.preco)).group_by(Filme.genero).all()

# Buscar por id (exemplo: menor id)
primeiro_id = session.query(func.min(Filme.id)).scalar()
filme_por_id = session.query(Filme).get(primeiro_id)

existe_ficcao = session.query(session.query(Filme).filter(Filme.genero == "Ficcao").exists()).scalar()

print("Total de filmes:", total_filmes)
print("Preço médio:", round(preco_medio, 2))
print("Preço médio por gênero:", [(g, round(p,2)) for g,p in preco_medio_por_genero])
print("Filme por id:", (filme_por_id.id, filme_por_id.titulo) if filme_por_id else None)
print("Existe ficção?:", existe_ficcao)


Total de filmes: 56
Preço médio: 18.71
Preço médio por gênero: [('Animação', 14.65), ('Aventura', 23.9), ('Ação', 20.65), ('Comédia', 14.4), ('Drama', 18.64), ('Fantasia', 27.9), ('Ficção', 21.54), ('Guerra', 19.43), ('Musical', 16.4), ('Romance', 14.6), ('Suspense', 18.4)]
Filme por id: (1, 'Interstellar')
Existe ficção?: False


/tmp/ipykernel_27613/614125963.py:9: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  filme_por_id = session.query(Filme).get(primeiro_id)



### 6) Atualização e remoção (enunciado)

- Aumente em 10% o preço dos filmes de **Ação** lançados entre **2000 e 2010** (inclusive) e confirme com uma listagem desses registros após a atualização.  
- Remova todos os filmes com **avaliação < 6.5** e confirme que **0** registros permanecem nesse critério.


In [9]:

# === GABARITO: Atualização e remoção ===
# Aumento 10% para Ação (2000..2010)
alvo = session.query(Filme).filter(
    (Filme.genero == "Acao") & (Filme.ano >= 2000) & (Filme.ano <= 2010)
).all()

for f in alvo:
    f.preco = round(f.preco * 1.10, 2)
session.commit()

print("Após aumento (Ação 2000..2010):", [(f.titulo, f.preco) for f in alvo])

# Remover avaliação < 6.5
apagados = session.query(Filme).filter(Filme.avaliacao < 6.5).delete(synchronize_session=False)
session.commit()

restantes_abaixo = session.query(Filme).filter(Filme.avaliacao < 6.5).count()
print("Removidos:", apagados, "| Restantes < 6.5:", restantes_abaixo)


Após aumento (Ação 2000..2010): []
Removidos: 2 | Restantes < 6.5: 0


---

## Parte 2 — Empresa (Relacionamentos)


### 7) Modelo relacional e criação (enunciado)

- Departamento: id (PK), nome (único). Rel.: funcionarios (1:N).  
- Funcionario: id, nome, cargo, salario, departamento_id (FK). Rel.: departamento (N:1), projetos (1:N).  
- Projeto: id, nome (único), descricao, funcionario_id (FK). Rel.: funcionario (N:1).  

Crie o banco/tabelas e a sessão.


In [11]:

# === GABARITO: Modelo relacional e criação ===
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

EMPRESA_DB_PATH = "empresa.db"
engine_emp = create_engine(f"sqlite:///{EMPRESA_DB_PATH}", echo=False)

BaseEmp = declarative_base()

class Departamento(BaseEmp):
    __tablename__ = "departamentos"
    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    funcionarios = relationship("Funcionario", back_populates="departamento")

class Funcionario(BaseEmp):
    __tablename__ = "funcionarios"
    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False)
    cargo = Column(String, nullable=False)
    salario = Column(Integer, nullable=False)
    departamento_id = Column(Integer, ForeignKey("departamentos.id"))
    departamento = relationship("Departamento", back_populates="funcionarios")
    projetos = relationship("Projeto", back_populates="funcionario", cascade="save-update, merge")

class Projeto(BaseEmp):
    __tablename__ = "projetos"
    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    descricao = Column(String)
    funcionario_id = Column(Integer, ForeignKey("funcionarios.id"))
    funcionario = relationship("Funcionario", back_populates="projetos")

BaseEmp.metadata.drop_all(engine_emp)  # garantir idempotência
BaseEmp.metadata.create_all(engine_emp)

SessionEmp = sessionmaker(bind=engine_emp)
sess_emp = SessionEmp()

print("Banco criado em:", EMPRESA_DB_PATH)


Banco criado em: empresa.db



### 8) Inserções e vínculos (enunciado)

Inserir no banco os seguintes dados referentes às respectivas tabelas:  
- Departamentos: TI, RH, Financeiro.  
- Funcionários: Carlos–TI (Desenvolvedor, 5000); Ana–RH (Analista de RH, 4000); Mariana–Financeiro (Contadora, 4500).  
- Projetos: Sistema Web–Carlos; Treinamento Interno–Ana; Auditoria 2024–Ana; Gestão de clientes–Mariana.  
Valide rapidamente com consultas simples.


In [12]:

# === GABARITO: Inserções e vínculos ===
ti = Departamento(nome="TI")
rh = Departamento(nome="RH")
fin = Departamento(nome="Financeiro")
sess_emp.add_all([ti, rh, fin])
sess_emp.commit()

carlos = Funcionario(nome="Carlos", cargo="Desenvolvedor", salario=5000, departamento=ti)
ana = Funcionario(nome="Ana", cargo="Analista de RH", salario=4000, departamento=rh)
mariana = Funcionario(nome="Mariana", cargo="Contadora", salario=4500, departamento=fin)
sess_emp.add_all([carlos, ana, mariana])
sess_emp.commit()

p1 = Projeto(nome="Sistema Web", descricao="Desenvolvimento de um sistema web", funcionario=carlos)
p2 = Projeto(nome="Treinamento Interno", descricao="Organização de treinamentos", funcionario=ana)
p3 = Projeto(nome="Auditoria 2024", descricao="Planejamento da auditoria anual", funcionario=ana)
p4 = Projeto(nome="Gestão de clientes", descricao="Controle de clientes", funcionario=mariana)
sess_emp.add_all([p1, p2, p3, p4])
sess_emp.commit()

print("Departamentos:", [d.nome for d in sess_emp.query(Departamento).all()])
print("Funcionários:", [(f.nome, f.departamento.nome) for f in sess_emp.query(Funcionario).all()])
print("Projetos:", [(p.nome, p.funcionario.nome) for p in sess_emp.query(Projeto).all()])


Departamentos: ['TI', 'RH', 'Financeiro']
Funcionários: [('Carlos', 'TI'), ('Ana', 'RH'), ('Mariana', 'Financeiro')]
Projetos: [('Sistema Web', 'Carlos'), ('Treinamento Interno', 'Ana'), ('Auditoria 2024', 'Ana'), ('Gestão de clientes', 'Mariana')]



### 9) Consultas com relacionamentos (joins) (enunciado)

- Liste funcionários com: nome, cargo, departamento e projetos (nome + descrição).  
- Liste departamentos com: nome do departamento, funcionários (nome, cargo, salário).  
- Liste projetos com: nome do projeto, funcionário responsável, departamento do responsável.  
- Inclua ao menos um filtro (ex.: por departamento) e uma ordenação (ex.: salário decrescente).  
- Evite N+1 ao listar funcionários e seus projetos (use carregamento antecipado).


In [13]:
# === GABARITO: Consultas com relacionamentos (sem joinedload) ===

print("Funcionários e seus projetos:")
funcs = sess_emp.query(Funcionario).all()
for f in funcs:
    print(f"- {f.nome} | {f.cargo} | Dep.: {f.departamento.nome}")
    for p in f.projetos:
        print(f"   • Projeto: {p.nome} — {p.descricao}")

print("\nDepartamentos e seus funcionários:")
deps = sess_emp.query(Departamento).all()
for d in deps:
    print(f"- {d.nome}")
    for f in d.funcionarios:
        print(f"   • {f.nome} | {f.cargo} | Salário: {f.salario}")

print("\nProjetos com responsável e departamento:")
projs = (sess_emp.query(Projeto)
         .join(Projeto.funcionario)
         .join(Funcionario.departamento)
         .order_by(Funcionario.salario.desc())
         .all())
for p in projs:
    print(f"- {p.nome} | Resp.: {p.funcionario.nome} | Dep.: {p.funcionario.departamento.nome}")


Funcionários e seus projetos:
- Carlos | Desenvolvedor | Dep.: TI
   • Projeto: Sistema Web — Desenvolvimento de um sistema web
- Ana | Analista de RH | Dep.: RH
   • Projeto: Treinamento Interno — Organização de treinamentos
   • Projeto: Auditoria 2024 — Planejamento da auditoria anual
- Mariana | Contadora | Dep.: Financeiro
   • Projeto: Gestão de clientes — Controle de clientes

Departamentos e seus funcionários:
- TI
   • Carlos | Desenvolvedor | Salário: 5000
- RH
   • Ana | Analista de RH | Salário: 4000
- Financeiro
   • Mariana | Contadora | Salário: 4500

Projetos com responsável e departamento:
- Sistema Web | Resp.: Carlos | Dep.: TI
- Gestão de clientes | Resp.: Mariana | Dep.: Financeiro
- Treinamento Interno | Resp.: Ana | Dep.: RH
- Auditoria 2024 | Resp.: Ana | Dep.: RH



### 10) Agregações envolvendo relacionamentos (enunciado)

- Quantidade de funcionários por departamento.  
- Quantidade de projetos por funcionário.  
- Quantidade de projetos por departamento.  
- Salário médio por departamento.  
- Liste “funcionários sem projetos” (nome do funcionário e seu departamento).  
Observação: utilize junções e funções de agregação apropriadas.


In [19]:

# === GABARITO: Agregações com relacionamentos ===

# Funcionários por departamento
funcs_por_dep = (sess_emp.query(Departamento.nome, func.count(Funcionario.id))
                 .join(Departamento.funcionarios)
                 .group_by(Departamento.id)
                 .all())

# Projetos por funcionário
proj_por_func = (sess_emp.query(Funcionario.nome, func.count(Projeto.id))
                 .outerjoin(Funcionario.projetos)
                 .group_by(Funcionario.id)
                 .all())

# Projetos por departamento (via funcionário)
proj_por_dep = (sess_emp.query(Departamento.nome, func.count(Projeto.id))
                .join(Departamento.funcionarios)
                .outerjoin(Funcionario.projetos)
                .group_by(Departamento.id)
                .all())

# Salário médio por departamento
sal_medio_dep = (sess_emp.query(Departamento.nome, func.avg(Funcionario.salario))
                 .join(Departamento.funcionarios)
                 .group_by(Departamento.id)
                 .all())

# Funcionários sem projetos
sem_projeto = (sess_emp.query(Funcionario)
               .outerjoin(Funcionario.projetos)
               .filter(Projeto.id.is_(None))
               .all())

print("Funcionários por departamento:", funcs_por_dep)
print("Projetos por funcionário:", proj_por_func)
print("Projetos por departamento:", proj_por_dep)
print("Salário médio por departamento:", [(d, round(m,2)) for d,m in sal_medio_dep])
print("Funcionários sem projetos:", [(f.nome, f.departamento.nome) for f in sem_projeto])


Funcionários por departamento: [('TI', 1), ('RH', 1), ('Financeiro', 1)]
Projetos por funcionário: [('Carlos', 1), ('Ana', 2), ('Mariana', 1)]
Projetos por departamento: [('TI', 1), ('RH', 2), ('Financeiro', 1)]
Salário médio por departamento: [('TI', 5000.0), ('RH', 4000.0), ('Financeiro', 4500.0)]
Funcionários sem projetos: []



### 11) Atualizações com base em relacionamentos (enunciado)

- Aumento de 10% no salário de todos os funcionários de um departamento escolhido (ex.: “TI”).  
- Transferência de todos os funcionários do departamento “TI” para “RH”.  
- Renomear um departamento (ex.: “RH” → “Pessoas & Cultura”).  
- Reatribuir um projeto de um funcionário para outro (ex.: mover “Auditoria 2024”).  
- Confirme cada mudança com listagens objetivas.


In [20]:

# === GABARITO: Atualizações com relacionamentos ===
dep_ti = sess_emp.query(Departamento).filter_by(nome="TI").first()
dep_rh = sess_emp.query(Departamento).filter_by(nome="RH").first()

# Aumento 10% no departamento TI
if dep_ti:
    funcs_ti = sess_emp.query(Funcionario).filter_by(departamento_id=dep_ti.id).all()
    for f in funcs_ti:
        f.salario = int(round(f.salario * 1.10))
    sess_emp.commit()

# Transferir todos de TI -> RH
if dep_ti and dep_rh:
    funcs_ti = sess_emp.query(Funcionario).filter_by(departamento_id=dep_ti.id).all()
    for f in funcs_ti:
        f.departamento = dep_rh
    sess_emp.commit()

# Renomear RH -> Pessoas & Cultura
dep_rh = sess_emp.query(Departamento).filter_by(nome="RH").first()
if dep_rh:
    dep_rh.nome = "Pessoas & Cultura"
    sess_emp.commit()

# Reatribuir projeto "Auditoria 2024" para Carlos
proj = sess_emp.query(Projeto).filter_by(nome="Auditoria 2024").first()
carlos = sess_emp.query(Funcionario).filter_by(nome="Carlos").first()
if proj and carlos:
    proj.funcionario = carlos
    sess_emp.commit()

# Listagens de conferência
print("Departamentos:", [d.nome for d in sess_emp.query(Departamento).all()])
print("Funcionários:", [(f.nome, f.cargo, f.salario, f.departamento.nome) for f in sess_emp.query(Funcionario).all()])
print("Projetos:", [(p.nome, p.funcionario.nome) for p in sess_emp.query(Projeto).all()])


Departamentos: ['TI', 'Pessoas & Cultura', 'Financeiro']
Funcionários: [('Carlos', 'Desenvolvedor', 5500, 'Pessoas & Cultura'), ('Ana', 'Analista de RH', 4000, 'Pessoas & Cultura'), ('Mariana', 'Contadora', 4500, 'Financeiro')]
Projetos: [('Sistema Web', 'Carlos'), ('Treinamento Interno', 'Ana'), ('Auditoria 2024', 'Carlos'), ('Gestão de clientes', 'Mariana')]


### 12) Backup e limpeza do banco (export → drop_all → reset)

**Objetivo:** realizar backup simples em CSV e, em seguida, limpar o banco.

**Tarefas:**
- **Exportar para CSV** (backup antes da limpeza):
  - `departamentos.csv`: `id, nome`
  - `funcionarios.csv`: `id, nome, cargo, salario, departamento_id`
  - `projetos.csv`: `id, nome, descricao, funcionario_id`
- **Validar backup**: verificar a **existência** dos 3 arquivos e o **número de linhas** de cada um.
- **Limpeza parcial**: executar `drop_all` para **excluir todas as tabelas**, **sem remover** o arquivo do banco.
- **Reset completo**: **remover o arquivo SQLite** (ex.: `empresa.db`) e confirmar que foi apagado.


In [21]:
# === GABARITO: 12) Backup e limpeza do banco (export → drop_all → reset) ===
import csv
from pathlib import Path

# Pastas/arquivos
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

DEP_CSV = DATA_DIR / "departamentos.csv"
FUNC_CSV = DATA_DIR / "funcionarios.csv"
PROJ_CSV = DATA_DIR / "projetos.csv"

# ---------- Exportar para CSV ----------
def export_csv(path, header, rows):
    with path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(header)
        for r in rows:
            w.writerow(r)

# Departamentos: id, nome
deps = sess_emp.query(Departamento.id, Departamento.nome).all()
export_csv(DEP_CSV, ["id", "nome"], deps)

# Funcionarios: id, nome, cargo, salario, departamento_id
funcs = sess_emp.query(
    Funcionario.id, Funcionario.nome, Funcionario.cargo,
    Funcionario.salario, Funcionario.departamento_id
).all()
export_csv(FUNC_CSV, ["id", "nome", "cargo", "salario", "departamento_id"], funcs)

# Projetos: id, nome, descricao, funcionario_id
projs = sess_emp.query(
    Projeto.id, Projeto.nome, Projeto.descricao, Projeto.funcionario_id
).all()
export_csv(PROJ_CSV, ["id", "nome", "descricao", "funcionario_id"], projs)

# ---------- Validar backup ----------
def count_data_rows(csv_path: Path) -> int:
    if not csv_path.exists():
        return 0
    # desconta o cabeçalho
    with csv_path.open("r", encoding="utf-8") as f:
        return max(sum(1 for _ in f) - 1, 0)

print("[Backup] Arquivos gerados?")
print(" - departamentos.csv:", DEP_CSV.exists(), "| linhas (dados):", count_data_rows(DEP_CSV))
print(" - funcionarios.csv :", FUNC_CSV.exists(),  "| linhas (dados):", count_data_rows(FUNC_CSV))
print(" - projetos.csv     :", PROJ_CSV.exists(),  "| linhas (dados):", count_data_rows(PROJ_CSV))

# ---------- Limpeza parcial: drop_all (sem remover o arquivo do banco) ----------
BaseEmp.metadata.drop_all(bind=engine_emp)
print("[Limpeza parcial] Tabelas removidas com drop_all (arquivo do banco preservado).")
print("Arquivo do banco ainda existe?", Path(EMPRESA_DB_PATH).exists())

# ---------- Reset completo: remover o arquivo SQLite ----------
# Fechar sessão e soltar conexões para liberar o arquivo no SO
sess_emp.close()
engine_emp.dispose()

db_path = Path(EMPRESA_DB_PATH)
if db_path.exists():
    db_path.unlink()
    print("[Reset completo] Arquivo do banco removido:", EMPRESA_DB_PATH)
else:
    print("[Reset completo] Arquivo já não existia:", EMPRESA_DB_PATH)

[Backup] Arquivos gerados?
 - departamentos.csv: True | linhas (dados): 3
 - funcionarios.csv : True | linhas (dados): 3
 - projetos.csv     : True | linhas (dados): 4
[Limpeza parcial] Tabelas removidas com drop_all (arquivo do banco preservado).
Arquivo do banco ainda existe? True
[Reset completo] Arquivo do banco removido: data/empresa.db
